In [1]:
from transformers import AutoTokenizer
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/Users/jashdalvi/miniforge3/envs/ml/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
train_prompt_file = "../data/prompts_train.csv"
train_summary_file = "../data/summaries_train.csv"
pdf = pd.read_csv(train_prompt_file)
sdf = pd.read_csv(train_summary_file)
df = pdf.merge(sdf, on="prompt_id")

In [4]:
id2fold = {
            "39c16e": 0,
            "814d6b": 1,
            "3b9047": 2,
            "ebad26": 3,
        }
df["fold"] = df["prompt_id"].map(id2fold)
df.head()

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,text,content,wording,corrected_text,fold
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00791789cc1f,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415,One element of an ideal tragedy is that it sho...,0
1,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0086ef22de8f,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058,The three elements of an ideal tragedy are Ha...,0
2,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0094589c7a22,Aristotle states that an ideal tragedy should ...,-0.387791,-0.584181,Aristotle states that an ideal tragedy should ...,0
3,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00cd5736026a,One element of an Ideal tragedy is having a co...,0.088882,-0.594710,One element of an ideal tragedy is having a co...,0
4,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00d98b8ff756,The 3 ideal of tragedy is how complex you need...,-0.687288,-0.460886,The 3 ideal of tragedy is how complex you need...,0


In [5]:
tokenizer(tokenizer.sep_token)

{'input_ids': [1, 2, 2], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [6]:
columns = [ "text", "prompt_title", "prompt_question", "prompt_text"]
fold_length_dict = {k:[] for k in range(4)}

for i, row in tqdm(df.iterrows(), total = len(df)):
    text_to_tokenize = f" {tokenizer.sep_token} ".join([row[col] for col in columns])
    tokens = tokenizer(text_to_tokenize)["input_ids"]
    fold_length_dict[int(row["fold"])].append(len(tokens))

  2%|▏         | 113/7165 [00:00<00:12, 578.17it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 7165/7165 [00:14<00:00, 510.28it/s]


In [7]:
for k, v in fold_length_dict.items():
    print(f"Fold : {k}")
    print(f"Mean of tokens: {np.mean(v)}")
    print(f"Median of tokens: {np.median(v)}")
    print(f"Max length of tokens: {np.max(v)}")
    print(f"Min length of tokens: {np.min(v)}")

Fold : 0
Mean of tokens: 814.0636849781234
Median of tokens: 798.0
Max length of tokens: 1481
Min length of tokens: 767
Fold : 1
Mean of tokens: 778.9963735267453
Median of tokens: 763.0
Max length of tokens: 1162
Min length of tokens: 723
Fold : 2
Mean of tokens: 790.1393728222996
Median of tokens: 767.0
Max length of tokens: 1508
Min length of tokens: 715
Fold : 3
Mean of tokens: 1257.1728456913827
Median of tokens: 1237.0
Max length of tokens: 1790
Min length of tokens: 1190
